In [ ]:
# written by Talia Blum
from ipywidgets import interact, interactive, fixed, interact_manual, interactive_output
import ipywidgets as widgets
import math
from IPython.display import display
import matplotlib.pyplot as plt
import numpy as np
import itertools

In [ ]:
def jeffersons_method(state_pops, num_seats, tot_pop):
    """
    returns a list of the number of seats for each state by rounding down
    params: 
        state_pops: list of populations of states
        tot_pop: the populations of each 
        num_seats: the goal number of seats
    return:
        list of seats for each state corresponding to state_pops list
    """
    return [math.floor(x / (tot_pop / num_seats)) for x in state_pops]

def adams_method(state_pops, num_seats, tot_pop):
    return [math.ceil(x / (tot_pop / num_seats)) for x in state_pops]

def websters_method(state_pops, num_seats, tot_pop):
    return [round(x / (tot_pop / num_seats)) for x in state_pops]

def huntington_hill_method(state_pops, num_seats, tot_pop):
    quotas = []
    for x in state_pops:
        quota = x / (tot_pop / num_seats)
        n = math.floor(quota)
        geom_mean = math.sqrt(n * (n+1))
        if quota < geom_mean:
            quotas.append(n)
        else:
            quotas.append(n + 1)
    return quotas

def plot_histogram(method, state_pops, num_seats, tot_pop):
    quotas = method(state_pops, num_seats, tot_pop)

    fig = plt.figure()
    ax = fig.add_axes([0,0,1,1])
    plt.ylim(0, num_seats+10)
    ax.axhline(num_seats,color="lightgreen",linewidth=2)
    plt.xlabel("states")
    plt.ylabel("number of seats")
    plt.title(method.__name__)
    ax.bar([chr(i) for i in range(ord('A'),ord('A')+len(quotas))] + ["total seats"], 
           quotas + [sum(quotas)])
    plt.show()
    return

def plot_histograms(methods, state_pops, num_seats, ideal_district_size):
    fig, axs = plt.subplots(1, 4, figsize=(20,4))
    tot_pop = ideal_district_size * num_seats
    
    for i in range(4):
        method = methods[i]
        quotas = method(state_pops, num_seats, tot_pop)

        #ax[i] = fig.add_axes([0,0,1,1])
        axs[i].set_ylim(0, max(num_seats+min(int(num_seats/10)+1, 5), sum(quotas)+10))
        axs[i].axhline(num_seats,color="lightgreen",linewidth=2)
        axs[i].set_xlabel("states")
        axs[i].set_ylabel("number of seats")
        axs[i].set_title(method.__name__)
        barlist = axs[i].bar([chr(i) for i in range(ord('A'),ord('A')+len(quotas))] + ["total seats"], 
               quotas + [sum(quotas)])
        if sum(quotas) == num_seats:
            barlist[len(quotas)].set_color('r')
        
        y = quotas + [sum(quotas)]
        for index, value in enumerate(y):
            axs[i].text(index, value,
                    str(value), color = 'blue', fontweight = 'bold')

    return


def get_int_input():
  while True:
    a = input()
    try:
      a = int(a)
      if a < 0:
        print("Please enter a positive integer.")
      else:
        return a
    except:
      print("Please enter an integer.") 

def input_state_pops():
    print("How many states are there?")
    num_states = get_int_input()
    print("How many seats do you want to apportion?")
    num_seats = get_int_input()
    state_pops = []
    print("Please enter the state populations one at a time:")
    for i in range(num_states):
        state_pops.append(get_int_input())
    return state_pops, num_seats
    

def main():
    #state_pops = [505,492,301]
    #num_seats = 100
    state_pops, num_seats = input_state_pops()
    tot_pop = sum(state_pops)
    ideal_district_size = tot_pop / num_seats
    
    print("Use the slider to artificially adjust the ideal district size (this is the total population divided by the number of seats!)")
    print("You can use your mouse, or the arrow keys on your keyboard to update the slider!")
    x = abs(ideal_district_size-1/num_seats) # change this to be slightly higher than what you actually need
    methods = [jeffersons_method, adams_method, websters_method, huntington_hill_method]
    quotas = interactive(plot_histograms,
                      methods=fixed(methods), 
                      state_pops=fixed(state_pops), 
                      num_seats=fixed(num_seats), 
                      ideal_district_size=widgets.FloatSlider(min=max(0,ideal_district_size-x), step = 1/num_seats, max=ideal_district_size+x, value=ideal_district_size))
    quotas.layout.height = '300px'
    display(quotas)

In [ ]:
main()